In [1]:
from collections.abc import Iterator, Sequence
import multiprocessing
import os
import typing
from typing import Protocol, SupportsIndex, TypeVar

import jax
import jax.numpy as jnp
import lerobot.common.datasets.lerobot_dataset as lerobot_dataset
import numpy as np
import torch

import openpi.models.model as _model
import openpi.training.config as _config
import openpi.transforms as _transforms
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
from pathlib import Path

# class AgiBotDataset(LeRobotDataset):
#     @classmethod
#     def create(
#         cls,
#         repo_id: str,
#         fps: int,
#         features: dict,
#         root: str | Path | None = None,
#         robot_type: str | None = None,
#         use_videos: bool = False,
#         tolerance_s: float = 1e-4,
#         image_writer_processes: int = 0,
#         image_writer_threads: int = 0,
#         video_backend: str | None = None,
#         image_path: str | None = None
#     ) -> "LeRobotDataset":
#         """Create a agibotworld LeRobot Dataset from scratch in order to record data."""
#         obj = cls.__new__(cls)
#         obj.meta = LeRobotDatasetMetadata.create(
#             repo_id=repo_id,
#             fps=fps,
#             robot_type=robot_type,
#             features=features,
#             root=root,
#             use_videos=use_videos
#         )
#         obj.repo_id = obj.meta.repo_id
#         obj.root = obj.meta.root
#         obj.revision = None
#         obj.tolerance_s = tolerance_s
#         obj.image_writer = None

#         if image_writer_processes or image_writer_threads:
#             obj.start_image_writer(image_writer_processes, image_writer_threads)

#         obj.episode_buffer = obj.create_episode_buffer()

#         obj.episodes = None
#         obj.hf_dataset = obj.create_hf_dataset()
#         obj.image_transforms = None
#         obj.delta_timestamps = None
#         obj.delta_indices = None
#         obj.episode_data_index = None
#         obj.image_path = Path(image_path) if image_path else None
#         return obj

#     def __getitem__(self, idx: int) -> dict:
#         """Override the parent __getitem__ to include image loading from file paths."""
#         # Get the base item from parent class
#         item = super().__getitem__(idx)
        
#         # Load images from file paths and add to item
#         image_data = self._query_images(None, idx)
        
#         # Map the loaded images to the expected format
#         if image_data:
#             if "observation.images" not in item:
#                 item["observation.images"] = {}
            
#             # Map camera names to expected keys
#             camera_mapping = {
#                 "observation.images.head_color": "head",
#                 "observation.images.hand_left_color": "hand_left", 
#                 "observation.images.hand_right_color": "hand_right"
#             }
            
#             for camera_key, image_tensor in image_data.items():
#                 if camera_key in camera_mapping:
#                     expected_key = camera_mapping[camera_key]
#                     item["observation.images"][expected_key] = image_tensor
        
#         return item

#     def _query_images(self, query_indices: dict[str, list[int]] | None, idx: int) -> dict[str, torch.Tensor]:
#         """Load images directly from file paths constructed from observation.images.path data."""
#         item = {}

#         # Check if we have observation.images.path in the dataset
#         if "observation.images.path" not in self.hf_dataset.column_names:
#             return item

#         if query_indices is not None:
#             # Load multiple images for delta timestamps
#             if "observation.images.path" in query_indices:
#                 selected_data = self.hf_dataset.select(query_indices["observation.images.path"])
#                 path_arrays = selected_data["observation.images.path"]

#                 all_images = {}
#                 for camera_name in ["hand_left_color", "hand_right_color", "head_color"]:
#                     images = []
#                     for path_array in path_arrays:
#                         img_path = self._construct_image_path(path_array, camera_name)
#                         image = self._load_single_image(img_path)
#                         images.append(image)
#                     all_images[f"observation.images.{camera_name}"] = torch.stack(images)
#                 item.update(all_images)
#         else:
#             # Load single image for current timestamp
#             path_array = self.hf_dataset[idx]["observation.images.path"]

#             # Load all camera images for current timestamp
#             for camera_name in ["hand_left_color", "hand_right_color", "head_color"]:
#                 img_path = self._construct_image_path(path_array, camera_name)
#                 item[f"observation.images.{camera_name}"] = self._load_single_image(img_path)
#         item['prompt'] = " "
#         return item

#     def _construct_image_path(self, path_array: list | torch.Tensor, camera_name: str) -> Path:
#         """Construct image path from path array and camera name."""
#         if isinstance(path_array, torch.Tensor):
#             path_array = path_array.tolist()

#         task_id, job_id, episode_id, frame_index = path_array
#         sn_code = "A2D0015AB00061"

#         extension = '.png' if camera_name.endswith('_depth') else '.jpg'

#         img_path = (self.image_path /
#                    str(task_id) /
#                    str(job_id) /
#                    sn_code /
#                    str(episode_id) /
#                    "camera" /
#                    str(frame_index) /
#                    f"{camera_name}{extension}")

#         return img_path
    
#     def _load_single_image(self, img_path: Path) -> torch.Tensor:
#         """Load a single image and convert to torch tensor."""
#         import PIL.Image

#         # Check if file exists
#         if not img_path.exists():
#             raise FileNotFoundError(f"Image file not found: {img_path}")

#         # Load image
#         image = PIL.Image.open(img_path).convert('RGB')

#         # Convert to numpy array and then to tensor [C, H, W] with values in [0, 1]
#         image_array = np.array(image, dtype=np.float32) / 255.0
#         image_tensor = torch.from_numpy(image_array).permute(2, 0, 1)

#         return image_tensor

#     def _query_hf_dataset(self, query_indices: dict[str, list[int]]) -> dict:
#         """Override to exclude image path from normal querying."""
#         return {
#             key: torch.stack(self.hf_dataset.select(q_idx)[key])
#             for key, q_idx in query_indices.items()
#             if key not in self.meta.video_keys and key != "observation.images.path"
#         }

In [2]:
import openpi.training.config as _config
_config.TrainConfig.image_path = "/dataset/SimDatas/"
config =  _config.TrainConfig
import dataclasses
import functools
import logging
import platform
from typing import Any

import etils.epath as epath
import flax.nnx as nnx
from flax.training import common_utils
import flax.traverse_util as traverse_util
import jax
import jax.experimental
import jax.numpy as jnp
import optax
import tqdm_loggable.auto as tqdm
import wandb

import openpi.models.model as _model
import openpi.shared.array_typing as at
import openpi.shared.nnx_utils as nnx_utils
import openpi.training.checkpoints as _checkpoints
import openpi.training.config as _config
import openpi.training.data_loader as _data_loader
import openpi.training.optimizer as _optimizer
import openpi.training.sharding as sharding
import openpi.training.utils as training_utils
import openpi.training.weight_loaders as _weight_loaders

In [3]:

import openpi.training.data_loader as _data_loader
config=_config.get_config("pi0_base_agibot_lora")
print(config)

TrainConfig(name='pi0_base_agibot_lora', project_name='openpi', exp_name=<tyro._singleton.PropagatingMissingType object at 0x78477814a410>, model=Pi0Config(action_dim=32, action_horizon=50, max_token_len=48, dtype='bfloat16', paligemma_variant='gemma_2b_lora', action_expert_variant='gemma_300m_lora'), weight_loader=CheckpointWeightLoader(params_path='s3://openpi-assets/checkpoints/pi0_base/params'), lr_schedule=CosineDecaySchedule(warmup_steps=1000, peak_lr=2.5e-05, decay_steps=30000, decay_lr=2.5e-06), optimizer=AdamW(b1=0.9, b2=0.95, eps=1e-08, weight_decay=1e-10, clip_gradient_norm=1.0), ema_decay=0.99, freeze_filter=All(PathRegex(pattern=re.compile('.*llm.*'), sep='/'), Not(PathRegex(pattern=re.compile('.*lora.*'), sep='/'))), data=LeRobotAgiBotDataConfig(repo_id='/dataset/lerobot_dataset', assets=AssetsConfig(assets_dir=None, asset_id=None), base_config=DataConfig(repo_id=None, asset_id=None, norm_stats=None, repack_transforms=Group(inputs=(), outputs=()), data_transforms=Group(in

In [4]:
# ! cd /dataset/SimDatas/ && ls -al

In [5]:
jax.config.update("jax_compilation_cache_dir", str(epath.Path("~/.cache/jax").expanduser()))

mesh = sharding.make_mesh(config.fsdp_devices)
data_sharding = jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec(sharding.DATA_AXIS))

data_loader = _data_loader.create_data_loader(
        config,
        sharding=data_sharding,
        num_workers=config.num_workers,
        shuffle=True,
    )


meta_data=LeRobotDatasetMetadata({
    Repository ID: '',
    Total episodes: '11',
    Total frames: '6032',
    Features: '['observation.images.path', 'observation.states', 'actions', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']',
})',



In [6]:
data_iter = iter(data_loader)
batch = next(data_iter)
logging.info(f"Initialized data loader:\n{training_utils.array_tree_to_info(batch)}")

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/.venv/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/.venv/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/.venv/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/app/src/openpi/training/data_loader.py", line 207, in __getitem__
    return self._transform(self._dataset[index])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/src/openpi/transforms.py", line 75, in __call__
    data = transform(data)
       